In [ ]:
# If PEPit is not installed yet, you can run this cell.
!pip install pepit==0.0.2

# Code description
This code aims at verifying (numerically) the first inequality from Theorem 1 of the paper
> "Last-Iterate Convergence of Optimistic Gradient Method for Monotone
       Variationnal Inequalities".

## Problem setup:
Consider the problem of finding a zero of a monotone Lipschitz operator:
       $$ \text{find } x \text{ such that }F(x) = 0$$
where $F$ is monotone and $L$-Lipschitz, and $Q$ is convex and compact.

## Algorithm: 
The past extragradient method is described by two sets of iterates:
$x^k$, $\tilde{x}^k$ where $k$ denotes the iteration counter, as follows:

- Initialize $\tilde{x}^0 = x^0$, and $x^1=x^0 - \gamma  F(x^0)$, then run
- for $k=1,...,N-1$:
     $$\tilde{x}^k    = x^k - \gamma  F(\tilde{x}^{k-1})$$
     $$ x^{k+1} = x^k - \gamma  F(\tilde{x}^k)$$

## Potential function:
Denotes $p_k := \|F(x^{k})\|^2 + 2 \| F(x^{k}) - F(\tilde{x}^{k-1}) \|^2$. The code compute the maximum (i.e., worst-case) value of
$$\|x^{k+1} - x^* \|^2 + A_{k+1} \gamma^2 p_{k+1} - \|x^{k}-x^*\|^2 - A_k  \gamma^2 p_k,$$
when $A_{k+1} = A_k + 1/3$. The expression should always be "$\cdot\leq 0$" for verifying the identity from Theorem 1 (with $\gamma\leq 1/3/L$ and $A_k\geq 32/3$). In the code below, we use k = 1 for notational convenience.

In [39]:
# Run this code before executing the cell below
from math import sqrt
import numpy as np
from PEPit import PEP
from PEPit.operators import LipschitzStronglyMonotoneOperator

In [42]:
##########################################################
# parameters: MODIFY HERE!

# pick the parameters for which you want to verify
# the inequality (numerically)
L = 1;
gamma = 1/3/L;
A1 = 100000; #too large values might make the solver bug

##########################################################

# verbose & verification tolerance options
verbose = 0;
tolerance = 1e-5;

# (0) Initialize an empty PEP
problem = PEP()

# (1) Set up the problem class
L  =  L; mu = 0; # F is 1-Lipschitz and 0-strongly monotone
F  = problem.declare_function(LipschitzStronglyMonotoneOperator, param={'L': L, 'mu': mu})
xs = F.stationary_point();  # x^* is a solution

# (2) Set up the starting points
tx0 = problem.set_initial_point() # this is \tilde{x}^0
x1  = problem.set_initial_point() # this is x^1

# (3) Run the algorithm
tx1     = x1 - gamma * F.gradient(tx0); 
x2      = x1 - gamma * F.gradient(tx1);

# (3) define the expressions (recall that our objective is to verify that
#     expression2 <= expression1 for all F and sequence generated by the
#     past extragradient method.
p1 = F.gradient(x1)**2 + 2 * ( F.gradient(x1) - F.gradient(tx0) )**2;
p2 = F.gradient(x2)**2 + 2 * ( F.gradient(x2) - F.gradient(tx1) )**2;
A2 = A1 + 1/3;
expression1 = ( x1-xs )**2 + A1 * gamma**2 * p1;
expression2 = ( x2-xs )**2 + A2 * gamma**2 * p2;

# (4) Set up the performance measure 
expression_to_verify = expression2 - expression1;
problem.set_performance_metric(expression_to_verify);

# (5) Solve the PEP
worstcase_value = problem.solve(verbose=verbose)


# (6) is the potential verified? Success if expression2 - expression1 <= 0 for the 
#     choice of the parameters above.
print('Did PEPit verify the potential (within prescribed numerical precision)? {:}  \t'.format(worstcase_value<tolerance))
worstcase_value

Did PEPit verify the potential (within prescribed numerical precision)? True  	


7.62021126060608e-09